# Проект: Принятие решений в бизнесе

## Описание проекта

Вы — аналитик крупного интернет-магазина. Вместе с отделом маркетинга вы подготовили список гипотез для увеличения выручки.
Приоритизируйте гипотезы, запустите A/B-тест и проанализируйте результаты.

### Описание данных

**Данные для первой части:**

Файл /datasets/hypothesis.csv

- Hypothesis — краткое описание гипотезы;
- Reach — охват пользователей по 10-балльной шкале;
- Impact — влияние на пользователей по 10-балльной шкале;
- Confidence — уверенность в гипотезе по 10-балльной шкале;
- Efforts — затраты ресурсов на проверку гипотезы по 10-балльной шкале. Чем больше значение Efforts, тем дороже проверка гипотезы.

**Данные для второй части:**

Файл /datasets/orders.csv

- transactionId — идентификатор заказа;
- visitorId — идентификатор пользователя, совершившего заказ;
- date — дата, когда был совершён заказ;
- revenue — выручка заказа;
- group — группа A/B-теста, в которую попал заказ.

Файл /datasets/visitors.csv. 

- date — дата;
- group — группа A/B-теста;
- visitors — количество пользователей в указанную дату в указанной группе A/B-теста

### Цель исследования

- Приоритизировать гипотезы
- Запуститить A/B-тест


### План работы

Часть 1. Приоритизация гипотез.
В файле /datasets/hypothesis.csv 9 гипотез по увеличению выручки интернет-магазина с указанными параметрами Reach, Impact, Confidence, Effort.
Задача
•	Примените фреймворк ICE для приоритизации гипотез. Отсортируйте их по убыванию приоритета.
•	Примените фреймворк RICE для приоритизации гипотез. Отсортируйте их по убыванию приоритета.
•	Укажите, как изменилась приоритизация гипотез при применении RICE вместо ICE. Объясните, почему так произошло.

Часть 2. Анализ A/B-теста
Вы провели A/B-тест и получили результаты, которые описаны в файлах /datasets/orders.csv и /datasets/visitors.csv.
Задача
Проанализируйте A/B-тест:
1.	Постройте график кумулятивной выручки по группам. Сделайте выводы и предположения.
2.	Постройте график кумулятивного среднего чека по группам. Сделайте выводы и предположения.
3.	Постройте график относительного изменения кумулятивного среднего чека группы B к группе A. Сделайте выводы и предположения.
4.	Постройте график кумулятивной конверсии по группам. Сделайте выводы и предположения.
5.	Постройте график относительного изменения кумулятивной конверсии группы B к группе A. Сделайте выводы и предположения.
6.	Постройте точечный график количества заказов по пользователям. Сделайте выводы и предположения.
7.	Посчитайте 95-й и 99-й перцентили количества заказов на пользователя. Выберите границу для определения аномальных пользователей.
8.	Постройте точечный график стоимостей заказов. Сделайте выводы и предположения.
9.	Посчитайте 95-й и 99-й перцентили стоимости заказов. Выберите границу для определения аномальных заказов.
10.	Посчитайте статистическую значимость различий в конверсии между группами по «сырым» данным. Сделайте выводы и предположения.
11.	Посчитайте статистическую значимость различий в среднем чеке заказа между группами по «сырым» данным. Сделайте выводы и предположения.
12.	Посчитайте статистическую значимость различий в конверсии между группами по «очищенным» данным. Сделайте выводы и предположения.
13.	Посчитайте статистическую значимость различий в среднем чеке заказа между группами по «очищенным» данным. Сделайте выводы и предположения.
14.	Примите решение по результатам теста и объясните его. Варианты решений: 
 1. Остановить тест, зафиксировать победу одной из групп.
 2. Остановить тест, зафиксировать отсутствие различий между группами.
 3. Продолжить тест.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import scipy.stats as stats
import datetime as dt
import numpy as np
import math
%matplotlib inline

In [ ]:
try:
    hypothesis = pd.read_csv('/datasets/hypothesis.csv')
    orders = pd.read_csv('/datasets/orders.csv')
    visitors = pd.read_csv('/datasets/visitors.csv')
except:
    hypothesis = pd.read_csv('hypothesis.csv')
    orders  =  pd.read_csv('orders.csv')
    visitors  =  pd.read_csv('visitors.csv')    

In [ ]:
def first_look(df):
    print('---------------------------Первые 5 строк----------------------------')
    display(df.head())
    print(' ')
    print(' ')
    print('--------------Общая информация--------------')
    print(' ')
    print(df.info())
    print(' ')
    print('-------------Пропуски------------- ')
    print(' ')
    count=0
    for element in df.columns:
        if df[element].isna().sum() > 0:
            print(element, ' - ', df[element].isna().sum(), 'пропусков')
            count = +1
    if count == 0:
        print(' ')
        print('Пропусков НЕТ')
    print(' ')
    print('-------------Дубликаты------------ ')
    print(' ')
    if df.duplicated().sum() > 0:
        print(' ')
        print('Дубликатов: ', df.duplicated().sum())
    else:
                print('Дубликатов НЕТ')
    print(' ')  
    

In [ ]:
pd.set_option('display.max_colwidth',500) #используем для полного просмотра названий гипотез
first_look(hypothesis)

In [ ]:
hypothesis

В таблице **hypothesis** - 9 гипотез по увеличению выручки интернет-магазина с указанными параметрами Reach, Impact, Confidence, Effort.

## Приоритизация гипотез

### Применим фреймворк ICE для приоритизации гипотез. Отсортируем их по убыванию приоритета.

**ICE = Impact x Confidence / Efforts**
- **Impact** — влияние
- **Confidence** — уверенность 
- **Effort** — трудозатраты

In [ ]:
hypothesis['ICE'] = hypothesis['Impact'] * hypothesis['Confidence'] / hypothesis['Efforts']
ice = hypothesis[['Hypothesis','ICE']].sort_values(by='ICE',ascending=False).round(2)
ice

### Применим фреймворк RICE для приоритизации гипотез. Отсортируем их по убыванию приоритета.

**RICE = (Reach x Impact x Confidence) / Efforts**
- **Reach** - охват
- **Impact** — влияние
- **Confidence** — уверенность 
- **Effort** — трудозатраты

In [ ]:
hypothesis['RICE'] = (hypothesis['Reach'] * hypothesis['Impact'] * hypothesis['Confidence']) / hypothesis['Efforts']
rice = hypothesis[['Hypothesis','RICE']].sort_values(by='RICE',ascending=False)
rice

In [ ]:
ice_rice = ice.merge(rice, on='Hypothesis')
ice_rice

In [ ]:
hypothesis.set_index('Hypothesis')[['ICE','RICE']].plot(kind='barh')

### Рассмотрим, как изменилась приоритизация гипотез при применении RICE вместо ICE.

При использовании модифицированного метода **RICE**, мы добавили компонент "Reach" (скольких пользователей затронет изменение, которое мы хотим внести), соответственно охват увеличил значимость влияния и уверенности, по отношению к затратам.

Приоритеты флейморка **ICE**:

- Запустить акцию, дающую скидку на товар в день рождения - Гипотеза №8
- Добавить два новых канала привлечения трафика, что позволит привлекать на 30% больше пользователей - Гипотеза №0
- Добавить форму подписки на все основные страницы, чтобы собрать базу клиентов для email-рассылок - Гипотеза №7 

Приоритеты флейморка **RICE**:

- Добавить форму подписки на все основные страницы, чтобы собрать базу клиентов для email-рассылок - Гипотеза №7
- Добавить блоки рекомендаций товаров на сайт интернет магазина, чтобы повысить конверсию и средний чек заказа - Гипотеза №2
- Добавить два новых канала привлечения трафика, что позволит привлекать на 30% больше пользователей - Гипотеза №0

Гипотезы под номерами "0" и "7" имеют приоритет в обоих методах.

## Анализ A/B-теста

### Проверим данные в orders и visitors

In [ ]:
first_look(orders)

In [ ]:
first_look(visitors)

In [ ]:
orders['date'] = pd.to_datetime(orders['date'])
orders.info()

In [ ]:
visitors['date'] = pd.to_datetime(visitors['date'])
visitors.info()

In [ ]:
sum_visitors_a = visitors[visitors['group']=='A']['visitors'].sum()
sum_visitors_b = visitors[visitors['group']=='B']['visitors'].sum()
print('Пользователей в группе А:', sum_visitors_a, 'Пользователей в группе В:',sum_visitors_b)

In [ ]:
unique_users = orders.groupby('visitorId').agg({'group':'nunique'})
unique_users.count()

In [ ]:
users_ab = unique_users.query('group >1')
users_ab.count()

In [ ]:
user_a = orders.query('group == "A"').count()
user_a

In [ ]:
user_b = orders.query('group == "B"').count()
user_b

Размер группы "А" сильно отличается от группы "В", что является недоработкой подготовки к тесту. Выявили 58 пользователей участвующих и в группе "А" и в группе "В", такого быть не должно, иначе придётся использовать уже совсем другие тесты, а они работают хуже, чем тесты для независимых выборок. Избавимся от них.

In [ ]:
users_ab = (np.intersect1d(orders.query('group == "A"')['visitorId']
                           .unique(),orders.query('group == "B"')['visitorId'].unique())) 

orders = orders[~orders['visitorId'].isin(users_ab)]

### Построим график кумулятивной выручки по группам. 

**Подготовим данные**

In [ ]:
# создаем массив уникальных пар значений дат и групп теста
datesGroups = orders[['date','group']].drop_duplicates() 

In [ ]:
# соберём агрегированные кумулятивные по дням данные о заказах
ordersAggregated = (datesGroups
                    .apply(lambda x: orders[np.logical_and(orders['date'] <= x['date'], orders['group'] == x['group'])]
                    .agg({'date' : 'max', 
                          'group' : 'max', 
                          'transactionId' : 'nunique', 
                          'visitorId' : 'nunique', 
                          'revenue' : 'sum'}), axis=1)
                    .sort_values(by=['date','group']))


In [ ]:
# получим агрегированные кумулятивные по дням данные о посетителях интернет-магазина
visitorsAggregated = (datesGroups
                      .apply(lambda x: visitors[np.logical_and(visitors['date'] <= x['date'], visitors['group'] == x['group'])]
                      .agg({'date' : 'max',
                            'group' : 'max',
                            'visitors' : 'sum'}), axis=1)
                      .sort_values(by=['date','group'])) 

In [ ]:
# объединим обе таблицы в одну с понятными названиями столбцов
cumulativeData = ordersAggregated.merge(visitorsAggregated, left_on=['date', 'group'], right_on=['date', 'group'])
cumulativeData.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']

print(cumulativeData.head(5)) 


**Построим графики кумулятивной выручки по дням и группам A/B-тестирования:**

In [ ]:
# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе А
cumulativeRevenueA = cumulativeData[cumulativeData['group']=='A'][['date','revenue', 'orders']]

# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе B
cumulativeRevenueB = cumulativeData[cumulativeData['group']=='B'][['date','revenue', 'orders']]

plt.figure(figsize=(12,7))
plt.title('Кумулятивная выручка по группам', fontsize=17)
plt.xlabel('Дата')
plt.ylabel('Выручка')
plt.grid()
# Строим график выручки группы А
plt.plot(cumulativeRevenueA['date'], cumulativeRevenueA['revenue'], label='A')

# Строим график выручки группы B
plt.plot(cumulativeRevenueB['date'], cumulativeRevenueB['revenue'], label='B')

plt.legend()


Выручка почти одинаково увеличивалась до середины августа. Во второй половине августа в группе "В" наблюдаются две точки резкого роста. Это может сигнализировать о всплесках числа заказов, либо о появлении очень дорогих заказов.

### Построим график кумулятивного среднего чека по группам

In [ ]:
plt.figure(figsize=(12,7))
plt.title('Кумулятивная выручка среднего чека по группам', fontsize=17)
plt.xlabel('Дата')
plt.ylabel('Выручка')
plt.grid()
# построим графики среднего чека по группам — разделим кумулятивную выручку на кумулятивное число заказов
plt.plot(cumulativeRevenueA['date'], cumulativeRevenueA['revenue']/cumulativeRevenueA['orders'], label='A')
plt.plot(cumulativeRevenueB['date'], cumulativeRevenueB['revenue']/cumulativeRevenueB['orders'], label='B')

plt.legend()

Кумулятивная выручка среднего чека стабилизируется в середине августа после роста у группы "А" и резко увеличивается у группы "В". Во второй половине августа у группы "А" небольшой стабильный рост, у группы "В" спад к концу месяца. Возможно, в группу "B" после 16 августа попали крупные заказы (резкий всплеск на графике)

### Построим график относительного изменения кумулятивного среднего чека группы B к группе A

In [ ]:
# собираем данные в одном датафрейме
mergedCumulativeRevenue = cumulativeRevenueA.merge(cumulativeRevenueB,
                                                   left_on='date', right_on='date', how='left', suffixes=['A', 'B'])

# cтроим отношение средних чеков
plt.figure(figsize=(12,7))
plt.grid()
plt.title('Относительное изменение кумулятивного среднего чека группы B к группе A', fontsize=17)
plt.plot(mergedCumulativeRevenue['date'], (mergedCumulativeRevenue['revenueB']/
                                           mergedCumulativeRevenue['ordersB'])/(mergedCumulativeRevenue['revenueA']/
                                                                                mergedCumulativeRevenue['ordersA'])-1)

# добавляем ось X
plt.axhline(y=0, color='red', linestyle='-') 

В нескольких точках графика есть резкие скачки, что также говорит о крупных заказах и выбросах.

### Построим график кумулятивной конверсии по группам

In [ ]:
# считаем кумулятивную конверсию
cumulativeData['conversion'] = cumulativeData['orders']/cumulativeData['visitors']

# отделяем данные по группе A
cumulativeDataA = cumulativeData[cumulativeData['group']=='A']

# отделяем данные по группе B
cumulativeDataB = cumulativeData[cumulativeData['group']=='B']

# строим графики
plt.figure(figsize=(12,7))
plt.title('Кумулятивная конверсия по группам', fontsize=17)
plt.xlabel('Дата')
plt.ylabel('Конверсия')
plt.grid()
plt.plot(cumulativeDataA['date'], cumulativeDataA['conversion'], label='A')
plt.plot(cumulativeDataB['date'], cumulativeDataB['conversion'], label='B')
plt.legend()

# задаем масштаб осей
plt.axis(["2019-08-01", "2019-08-31", 0.0238, 0.036]) 


Кумулятивная конверсия на графиках имеет небольшие скачки в начале месяца. После 10 августа стабилизируется в группе "А" ~ 2.6%, в группе "В" ~ 3%, что говорит о стабильной конверсии пользователей в покупателей. 

<br/>

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

<b>Все отлично!👍</b> 
    
Вывод верный!
</div>

<div class="alert alert-warning">
    <h2> Комментарий ревьюера <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации⚠️</b>
        
Давай немного подтюним график? 
        
Стоит ограничить значения по оси Y, чтобы мы видели динамику конверсии более детально. В качестве границ можно установить `min/max` значения конверсии + небольшая дельта для корректного отображения графика.
        
Попробуем?
</div>

<div class="alert alert-block alert-info">
Исправил <a class="tocSkip"></h2>

<br/>

<div class="alert alert-success">
<h2> Комментарий ревьюера v.2 <a class="tocSkip"> </h2>

<b>Все отлично!👍</b> 
    
То, что нужно! Отлично!

### Построим график относительного изменения кумулятивной конверсии группы B к группе A

In [ ]:
mergedCumulativeConversions = (cumulativeDataA[['date','conversion']]
                               .merge(cumulativeDataB[['date','conversion']],
                                      left_on='date', right_on='date', how='left', suffixes=['A', 'B']))

plt.figure(figsize=(12,7))
plt.title('Относительное изменение кумулятивной конверсии группы B к группе A', fontsize=17)
plt.xlabel('Дата')
plt.ylabel('Конверсия')
plt.grid()
plt.plot(mergedCumulativeConversions['date'],
         mergedCumulativeConversions['conversionB']/mergedCumulativeConversions['conversionA']-1,
         label="Относительный прирост конверсии группы B относительно группы A")
plt.legend()
plt.axhline(y=0, color='green', linestyle='--')
plt.axhline(y=-0.1, color='red', linestyle='--')
plt.axis(["2019-08-01", '2019-08-31', -0.3, 0.3]) 

В начале месяца мы видим колебания конверсии группы "В" относительно группы "А", но ~ с 6 августа конверсия группы "В" растёт до ~ 20%, снижаясь к концу месяца до 16%. Проанализируем аномалии.

<br/>

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

<b>Все отлично!👍</b> 
    
Отлично, нащупали аномалии!
</div>

### Построим точечный график количества заказов по пользователям

In [ ]:
ordersByUsers  = (orders.drop(['group', 'revenue', 'date'], 
                             axis=1).groupby('visitorId', as_index=False)
                  .agg({'transactionId': 'nunique'}))
ordersByUsers.columns = ['userId', 'orders']
plt.figure(figsize=(12,7))
plt.hist(ordersByUsers['orders'])
plt.grid()
plt.title('Распределение количества заказов по пользователям', fontsize=17)
plt.xlabel('Кол-во покупок')
plt.ylabel('Кол-во пользователей')

In [ ]:
print(ordersByUsers.sort_values(by='orders', ascending=False).head(10)) 

In [ ]:
ordersByUsers.describe()

In [ ]:
plt.figure(figsize=(12,7))
plt.grid()
x_values = pd.Series(range(0,len(ordersByUsers)))
plt.scatter(x_values, ordersByUsers['orders']) 
plt.title('Распределение количества заказов по пользователям', fontsize=17)
plt.ylabel('Кол-во покупок')
plt.xlabel('Кол-во пользователей')

Подавляющее большинство совершило 1 покупку, но также есть и с 2мя. Несколько аномалий в 3 покупки.

### Посчитаем 95-й и 99-й перцентили количества заказов на пользователя.
- **Выберем границу для определения аномальных пользователей**

In [ ]:
print(np.percentile(ordersByUsers['orders'], [95, 99])) 

Установим границу аномальных заказов в 1 покупку. 

<br/>

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

<b>Все отлично!👍</b> 
    
Отлично, всё верно!
</div>

### Построим точечный график стоимости заказов

In [ ]:
plt.figure(figsize=(12,7))
plt.xlabel('Количество заказов')
plt.ylabel('Стоимость заказа')
plt.title('График стоимости заказов', fontsize=17)
x_values = pd.Series(range(0,len(orders)))
plt.scatter(x_values, orders['revenue'])
plt.grid()

По точечному графику видно явный выброс более 1.2 млн и 0.2 млн.

### Посчитаем 95-й и 99-й перцентили стоимости заказов
- **Выберем границу для определения аномальных заказов**

In [ ]:
print(np.percentile(orders['revenue'], [95, 99]))

Не более 5% совершили покупку больше чем на 26785 и не более 1% больше чем на 53904. Установим 26785 границей для аномальной суммы заказа.

<br/>

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

<b>Все отлично!👍</b> 
    
И тут всё отлично!
</div>

### Посчитаем статистическую значимость различий в конверсии между группами по «сырым» данным

- **H₀ - статистически значимых различий в конверсии между группами нет** 
- **H₁ - статистически значимые различия в конверсии между группами есть**

<br/>

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

<b>Все отлично!👍</b> 
    
Гипотезы сформулированы верно!
</div>

In [ ]:
# подготовим данные
visitorsADaily = visitors[visitors['group'] == 'A'][['date', 'visitors']]
visitorsADaily.columns = ['date', 'visitorsPerDateA']

visitorsACummulative = visitorsADaily.apply(
    lambda x: visitorsADaily[visitorsADaily['date'] <= x['date']].agg(
        {'date': 'max', 'visitorsPerDateA': 'sum'}
    ),
    axis=1,
)
visitorsACummulative.columns = ['date', 'visitorsCummulativeA']

visitorsBDaily = visitors[visitors['group'] == 'B'][['date', 'visitors']]
visitorsBDaily.columns = ['date', 'visitorsPerDateB']

visitorsBCummulative = visitorsBDaily.apply(
    lambda x: visitorsBDaily[visitorsBDaily['date'] <= x['date']].agg(
        {'date': 'max', 'visitorsPerDateB': 'sum'}
    ),
    axis=1,
)
visitorsBCummulative.columns = ['date', 'visitorsCummulativeB']

ordersADaily = (
    orders[orders['group'] == 'A'][['date', 'transactionId', 'visitorId', 'revenue']]
    .groupby('date', as_index=False)
    .agg({'transactionId': pd.Series.nunique, 'revenue': 'sum'})
)
ordersADaily.columns = ['date', 'ordersPerDateA', 'revenuePerDateA']

ordersACummulative = ordersADaily.apply(
    lambda x: ordersADaily[ordersADaily['date'] <= x['date']].agg(
        {'date': 'max', 'ordersPerDateA': 'sum', 'revenuePerDateA': 'sum'}
    ),
    axis=1,
).sort_values(by=['date'])
ordersACummulative.columns = [
    'date',
    'ordersCummulativeA',
    'revenueCummulativeA',
]

ordersBDaily = (
    orders[orders['group'] == 'B'][['date', 'transactionId', 'visitorId', 'revenue']]
    .groupby('date', as_index=False)
    .agg({'transactionId': pd.Series.nunique, 'revenue': 'sum'})
)

ordersBDaily.columns = ['date', 'ordersPerDateB', 'revenuePerDateB']

ordersBCummulative = ordersBDaily.apply(
    lambda x: ordersBDaily[ordersBDaily['date'] <= x['date']].agg(
        {'date': 'max', 'ordersPerDateB': 'sum', 'revenuePerDateB': 'sum'}
    ),
    axis=1,
).sort_values(by=['date'])
ordersBCummulative.columns = [
    'date',
    'ordersCummulativeB',
    'revenueCummulativeB',
]

data = (
    ordersADaily.merge(
        ordersBDaily, left_on='date', right_on='date', how='left'
    )
    .merge(ordersACummulative, left_on='date', right_on='date', how='left')
    .merge(ordersBCummulative, left_on='date', right_on='date', how='left')
    .merge(visitorsADaily, left_on='date', right_on='date', how='left')
    .merge(visitorsBDaily, left_on='date', right_on='date', how='left')
    .merge(visitorsACummulative, left_on='date', right_on='date', how='left')
    .merge(visitorsBCummulative, left_on='date', right_on='date', how='left')
)

Описание таблицы **data**:
- date — дата;
- ordersPerDateA — количество заказов в выбранную дату в группе A;
- revenuePerDateA — суммарная выручка в выбранную дату в группе A;
- ordersPerDateB — количество заказов в выбранную дату в группе B;
- revenuePerDateB — суммарная выручка в выбранную дату в группе B;
- ordersCummulativeA — суммарное число заказов до выбранной даты включительно в группе A;
- revenueCummulativeA — суммарная выручка до выбранной даты включительно в группе A;
- ordersCummulativeB — суммарное количество заказов до выбранной даты включительно в группе B;
- revenueCummulativeB — суммарная выручка до выбранной даты включительно в группе B;
- visitorsPerDateA — количество пользователей в выбранную дату в группе A;
- visitorsPerDateB — количество пользователей в выбранную дату в группе B;
- visitorsCummulativeA — количество пользователей до выбранной даты включительно в группе A;
- visitorsCummulativeB — количество пользователей до выбранной даты включительно в группе B.

In [ ]:
display(data.head(5)) 

Создадим переменные ordersByUsersA и ordersByUsersB со столбцами ['userId', 'orders']. В них для пользователей, которые заказывали хотя бы 1 раз, укажем число совершённых заказов

In [ ]:
ordersByUsersA = (
    orders[orders['group'] == 'A']
    .groupby('visitorId', as_index=False)
    .agg({'transactionId': pd.Series.nunique})
)
ordersByUsersA.columns = ['userId', 'orders']

ordersByUsersB = (
    orders[orders['group'] == 'B']
    .groupby('visitorId', as_index=False)
    .agg({'transactionId': pd.Series.nunique})
)
ordersByUsersB.columns = ['userId', 'orders'] 

Объявим переменные sampleA и sampleB, в которых пользователям из разных групп будет соответствовать количество заказов. Тем, кто ничего не заказал, будут соответствовать нули. Это нужно, чтобы подготовить выборки к проверке критерием Манна-Уитни.

In [ ]:
pd.Series(0, index=np.arange(data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])), name='orders')
 
sampleA = (pd.concat([ordersByUsersA['orders'],
                      pd.Series(0, index=np.arange(data['visitorsPerDateA']
                                                   .sum() - len(ordersByUsersA['orders'])), name='orders')],axis=0))

sampleB = (pd.concat([ordersByUsersB['orders'],
                      pd.Series(0, index=np.arange(data['visitorsPerDateB']
                                                   .sum() - len(ordersByUsersB['orders'])), name='orders')],axis=0))

**Посчитаем статистическую значимость различия в конверсии между группами**

In [ ]:
print("p-value: {0:.5f}".format(stats.mannwhitneyu(sampleA, sampleB)[1]))

print("Относительный прирост группы B к группе A: {0:.2%}".format(sampleB.mean()/sampleA.mean()-1))

p-value < 0.05 - отвергаем нулевую гипотезу. Относительный прирост конверсии группы B = 15.98%

<br/>

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

<b>Все отлично!👍</b> 
    
Тут всё правильно - молодец, что пояснил, почему нужна особая подготовка!
</div>

### Посчитаем статистическую значимость различий в среднем чеке заказа между группами по «сырым» данным

- **H₀ - статистически значимых различий в среднем чеке заказа между группами нет** 
- **H₁ - статистически значимые различия в среднем чеке заказа между группами есть**

In [ ]:
print("p-value: {0:.3f}".format(stats.mannwhitneyu(orders[orders['group']=='A']['revenue'], 
                                          orders[orders['group']=='B']['revenue'])[1]))

print("Относительный прирост группы B к группе A: {0:.2%}".format(orders[orders['group']=='B']['revenue']
                       .mean()/orders[orders['group']=='A']['revenue'].mean()-1))

p-value > 0.05 - не отвергаем нулевую гипотезу. Относительный прирост различий в среднем чеке группы B = 28.66%

### Посчитаем статистическую значимость различий в конверсии между группами по «очищенным» данным

- **H₀ - статистически значимых различий в конверсии между группами по «очищенным» данным нет** 
- **H₁ - статистически значимые различия в конверсии между группами по «очищенным» данным есть**

- 95-й и 99-й перцентили средних чеков равны 26785 и 53904
- 95-й и 99-й перцентили количества заказов на одного пользователя равны 1 и 2

In [ ]:
# примем за аномальных пользователей тех, кто совершил больше 1 заказа или совершил заказ дороже 28 000 рублей
usersWithManyOrders = pd.concat(
    [
        ordersByUsersA[ordersByUsersA['orders'] > 1]['userId'],
        ordersByUsersB[ordersByUsersB['orders'] > 1]['userId'],
    ],
    axis=0,
)
usersWithExpensiveOrders = orders[orders['revenue'] > 26785]['visitorId']

# Объединим аномальных пользователей
abnormalUsers = (
    pd.concat([usersWithManyOrders, usersWithExpensiveOrders], axis=0)
    .drop_duplicates()
    .sort_values()
)
print(abnormalUsers.head(5))
print('Аномальных пользователей:', abnormalUsers.shape[0]) 

In [ ]:
# подготовим выборки количества заказов по пользователям по группам теста
sampleAFiltered = pd.concat(
    [
        ordersByUsersA[
            np.logical_not(ordersByUsersA['userId'].isin(abnormalUsers))
        ]['orders'],
        pd.Series(
            0,
            index=np.arange(
                data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])
            ),
            name='orders',
        ),
    ],
    axis=0,
)

sampleBFiltered = pd.concat(
    [
        ordersByUsersB[
            np.logical_not(ordersByUsersB['userId'].isin(abnormalUsers))
        ]['orders'],
        pd.Series(
            0,
            index=np.arange(
                data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders'])
            ),
            name='orders',
        ),
    ],
    axis=0,
)

# применим статистический критерий Манна-Уитни к полученным выборкам

print("p-value: {0:.3f}".format(stats.mannwhitneyu(sampleAFiltered, sampleBFiltered)[1]))
print("Относительный прирост группы B к группе A: {0:.2%}".format(sampleBFiltered.mean()/sampleAFiltered.mean()-1))

p-value < 0.05 - отвергаем нулевую гипотезу. Относительный прирост конверсии группы B = 17.39%

<br/>

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

<b>Все отлично!👍</b> 
    
Тут тоже всё хорошо!
</div>

### Посчитаем статистическую значимость различий в среднем чеке заказа между группами по «очищенным» данным

- **H₀ - статистически значимых различий в среднем чеке заказа между группами по «очищенным» данным нет** 
- **H₁ - статистически значимые различия в среднем чеке заказа между группами по «очищенным» данным есть**

In [ ]:
print("p-value: {0:.3f}".format(stats.mannwhitneyu(
    orders[np.logical_and(
        orders['group']=='A',
        np.logical_not(orders['visitorId'].isin(abnormalUsers)))]['revenue'],
    orders[np.logical_and(
        orders['group']=='B',
        np.logical_not(orders['visitorId'].isin(abnormalUsers)))]['revenue'])[1]))

print("Относительный прирост группы B к группе A: {0:.2%}".format(
    orders[np.logical_and(
        orders['group']=='B',np.logical_not(orders['visitorId'].isin(abnormalUsers)))]['revenue'].mean()/
    orders[np.logical_and(
        orders['group']=='A',
        np.logical_not(orders['visitorId'].isin(abnormalUsers)))]['revenue'].mean() - 1))



p-value > 0.05 - не отвергаем нулевую гипотезу. Относительная убыль различий в среднем чеке заказа по «очищенным» данным группы B = 3.37%

- **Присутствуют статистически значимые различия по конверсии между группами, как по «сырым», так и по "очищенным" данным**
- **Отсутствуют статистически значимые различия по среднему чеку между группами, как по «сырым», так и по "очищенным" данным**

<br/>

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

<b>Все отлично!👍</b> 
    
И здесь всё верно!
</div>

## Вывод:

**После применения фреймворков ICE и RICE для приоритизации гипотез, рекомендуется:**

- Добавить два новых канала привлечения трафика, что позволит привлекать на 30% больше пользователей - Гипотеза №0
- Добавить форму подписки на все основные страницы, чтобы собрать базу клиентов для email-рассылок - Гипотеза №7 



**На основании полученных данных, тест можно остановить, и признать победителем группу "В".** Различий в среднем чеке нет, но статистически значимые различия конверсии, как по "сырым", так и по "очищенным" данным в пользу группы "В". 

- Относительный прирост конверсии группы B = 15.98%
- Относительный прирост различий в среднем чеке группы B = 28.66%
- Относительный прирост конверсии по "очищенным" данным группы B = 17.39%

Единственный показатель, где группа "В" проиграла - Относительная убыль различий в среднем чеке заказа по «очищенным» данным = 3.37%

Графики 'Относительного изменения кумулятивной конверсии группы B к группе A', 'Относительного изменения кумулятивного среднего чека группы B к группе A', 'Кумулятивной выручки по группам', 'Кумулятивной выручки среднего чека по группам' также показывают лидерство группы "В".


<div style="border:solid Chocolate 2px; padding: 40px">

# Комментарий ревьюера: общий вывод по проекту.

Алексей, у тебя получился очень сильный и качественный проект - отличная работа!
    
Ты классно провёл анализ исходных данных, отлично применил визуализацию для выявления аномалий, применил верные инструменты для определения их границ. В части проверки статистических гипотез тоже нареканий нет - инструментарий выбран корректно, гипотезы сформулированы верно.
    
Мне понравилось оформление твоей работы - строгий и лаконичнйы стиль повествование, наличие структуры в работе, оглавление - выглядит как настоящий аналитический отчёт, здорово!
    
В работе я отметил несколько мест, требующих небольших правок, но все эти правки минорные - ты отлично справился с заданием, а правки доведут работу до совершенства.
    
Также оставляй свои комментарии, если у тебя есть какие-то вопросы - я обязательно на них отвечу!
    
Жду твою работу на повторном ревью.
    
Спасибо тебе за работу, и до встречи:)

<div style="border:solid Chocolate 2px; padding: 40px">

# Комментарий ревьюера: общий вывод по проекту v.2.

Алексей, теперь твоя работа идеальная - придраться не к чему даже при желании!
    
Спасибо за оперативные правки, принимаю твой проект.
    
Успехов в дальнейшем обучении!